In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy.signal import find_peaks
from tqdm import tqdm
antal_omegaer = 64
antal_A_ext = 64

omega_list = np.array(np.linspace(0.001, 3, antal_omegaer))
coupling_strengths = np.array(np.linspace(0.001, 0.2, antal_A_ext))
omega_list
coupling_strengths
# defining parameters from Krishna/Jensen

k_Nin = 5.4 # min^{-1}
k_lin = 0.018 # min^{-1} 
k_t = 1.03 # mu * M^{-1} * min^{-1}
k_tl = 0.24 # min^{-1}
K_I = 0.035 # mu * M
K_N = 0.029 # mu * M
gamma_m = 0.017 # min^{-1}
alpha = 1.05 # mu * M^{-1} * min^{-1}
N_tot = 1.0 # mu * M
k_a = 0.24 # min^{-1}
k_i = 0.18 # min^{-1}
k_p = 0.036 # min^{-1}
k_A20 = 0.0018 # mu * M
IKK_tot = 2.0 # mu * M
A20 = 0.0026 # mu * M
# defining time
time = (0, 5000)

k = 0.5

# definerer startparameters
N_n0, I_m0, I0, IKK_a0, IKK_i0, TNF_k = 1, 0.5, 0.5, 0.5, 0.5, k
y0 = [N_n0, I_m0, I0, IKK_a0, IKK_i0, TNF_k]
(3.6739411424841872, 9.612166599095634e-05)
# defining omega, number of oscillations, amplitude, TNF_0 and start parameters as the last values of the first simulation
omega = 1
no_osc = 200
Amp1 = 0.1
T_intt = 108.44444444444444
A_intt = 0.1293511878383227
p0 = [0.037770776771579556, 0.4050017925580332, 4.076546559955566, 0.1799999999999995, 1.5500000000000012]

T_extt = omega * T_intt
# defining equations from Krishna/Jensen
def N_n_change(t, N_n, I_m, I, IKK_a, IKK_i, TNF):
    dN_ndt = k_Nin * (N_tot - N_n) * K_I / (K_I + I) - k_lin * I * (N_n / (K_N + N_n))
    return dN_ndt

def I_m_change(t, N_n, I_m, I, IKK_a, IKK_i, TNF):
    dI_mdt = k_t * (N_n**2) - gamma_m * I_m
    return dI_mdt

def I_change(t, N_n, I_m, I, IKK_a, IKK_i, TNF):
    dIdt = k_tl * I_m - alpha * IKK_a * (N_tot - N_n) * I / (K_I + I)
    return dIdt

def IKK_a_change(t, N_n, I_m, I, IKK_a, IKK_i, TNF):
    dIKK_adt = k_a * TNF * (IKK_tot - IKK_a - IKK_i) - k_i * IKK_a
    return dIKK_adt

def IKK_i_change(t, N_n, I_m, I, IKK_a, IKK_i, TNF):
    dIKK_idt = k_i * IKK_a - k_p * IKK_i * k_A20 / (k_A20 + A20 * TNF)
    return dIKK_idt

def system_nfkb(t, y, Amp_new):
    # Extract current state variables, assume TNF is not part of y here
    N_n, I_m, I, IKK_a, IKK_i = y
    # Dynamically compute TNF based on current time t (this means it will be a constant passed to the next equations for this dt)
    TNF = k + Amp_new * np.sin(2 * np.pi * (1 / T_extt) * t)
    # calling differential equation functions, passing TNF as an argument
    # Updating function definitions accordingly if they require TNF
    return [N_n_change(t, *y, TNF), 
            I_m_change(t, *y, TNF), 
            I_change(t, *y, TNF), 
            IKK_a_change(t, *y, TNF), 
            IKK_i_change(t, *y, TNF)]
# defining function to oscillate TNF (sinusoidal)
def TNF_sin_osc(oscillations, T_int, Amp, OOmega, N_n0, I_m0, I0, IKK_a0, IKK_i0):

    # calculating the period of TNF in order to simulate with the desired ratio of external over internal period
    T_ext = OOmega * T_int
    omega = T_int / T_ext

    history = {"t":[] ,"N_n": [], "I_m": [], "I": [], "IKK_a":[], "IKK_i":[], "TNF":[]}

    
    for i in range(oscillations):
        if i == 0:
            state = N_n0, I_m0, I0, IKK_a0, IKK_i0
        else:
            state = history["N_n"][-1], history["I_m"][-1], history["I"][-1], history["IKK_a"][-1], history["IKK_i"][-1]

        sys = solve_ivp(system_nfkb, (0, T_ext), state, args=(Amp,), method='RK45', max_step=1, dense_output=True) # Is this step size too low?
        print(sys)
        N_n, I_m, I, IKK_a, IKK_i = sys["y"][0], sys["y"][1], sys["y"][2], sys["y"][3], sys["y"][4]

        new_t = np.array(sys["t"]) + (history["t"][-1] if history["t"] else 0)
        history["t"].extend(new_t)
        history["N_n"].extend(N_n)
        history["I_m"].extend(I_m)
        history["I"].extend(I)
        history["IKK_a"].extend(IKK_a)
        history["IKK_i"].extend(IKK_i)
        history["TNF"].extend(k + Amp * np.sin(np.pi * (1 / T_ext) * new_t))

    return history, omega, T_ext
# simulating system with sinus oscillations
TNF_sim2, ome, T_TNF2 = TNF_sin_osc(no_osc, T_intt, Amp1, omega, *p0)
# unpacking
t2, N_n2, I_m2, I2, IKK_a2, IKK_i2, TNF2 = np.array(TNF_sim2["t"]), np.array(TNF_sim2["N_n"]), np.array(TNF_sim2["I_m"]), np.array(TNF_sim2["I"]), np.array(TNF_sim2["IKK_a"]), np.array(TNF_sim2["IKK_i"]), np.array(TNF_sim2["TNF"])
len(t2)/15.642
peaks_test = find_peaks(TNF2)[0]
len(peaks_test)
plt.figure(figsize=(14,8))
x = 5
plt.plot(t2[1500:], x*N_n2[1500:], color="#FF0000", alpha=0.8, lw=0.5, label = f'{x}x  NF$\kappa$B')
plt.plot(t2[1500:], TNF2[1500:], alpha=1, color="#03FF00", lw=0.5, label = f'TNF with T = {T_TNF2:.4}')
plt.title(rf'Sinusoidal oscillation: $\Omega$ = {round(omega)}/1, A={Amp1}')
plt.xlabel('time [min.]')
plt.ylabel('concentration')
plt.legend()
def classify_entrianment(y_int, y_ext, current_tongue):
    window = 500
    # Apply a small window and require peak higher than mean
    peaks_internal = len(find_peaks(y_int[window:], height=np.mean(y_int))[0])
    peaks_external = len(find_peaks(y_ext[window:], height=np.mean(y_ext))[0])
    
    # 1/Omega because we are using frequency, which is 1/T
    Omega_ratio = peaks_internal/peaks_external

    print(Omega_ratio)

    rounded_ratio = np.round(Omega_ratio, 4)
    
    print(rounded_ratio)
    print(current_tongue - 1e-2, rounded_ratio, current_tongue + 1e-2)

    # Arbritrary threshold, we hope it works
    if current_tongue - 1e-2 <= rounded_ratio <= current_tongue + 1e-2:
        return True
    else:
        return False

Omega = classify_entrianment(N_n2, TNF2, 1)
print(Omega)
### Zooming in:
plt.figure(figsize=(14,8))
x = 5
plt.plot(t2[-6000:-4000], x*N_n2[-6000:-4000], color="#FF0000", alpha=0.8, lw=0.5, label = f'{x}x  NF$\kappa$B')
plt.plot(t2[-6000:-4000], TNF2[-6000:-4000], alpha=1, color="#03FF00", lw=0.5, label = f'TNF with T = {T_TNF2:.4}')
plt.title(rf'Sinusoidal oscillation: $\Omega$ = {round(omega)}/1, A={Amp1}')
plt.xlabel('time [min.]')
plt.ylabel('concentration')
plt.legend()
fig = plt.figure(figsize=(6, 6), dpi=200)
ax_3d = fig.add_subplot(111, projection='3d')  # Correct index for 3D plot
ax_3d.plot(I_m2, I2, N_n2 * 5, color="#FF0000", alpha=0.8, lw=0.5, label=f'System Trajectory with Omega = {omega}/1, A = {Amp1}')
ax_3d.set_xlabel(r'IkB_m', labelpad=10)
ax_3d.set_ylabel('IkB', labelpad=10)
# Adjust the z-label position
ax_3d.set_zlabel('5x NFkB', labelpad=10)  # Adjust labelpad as needed

# Adjusting the view angle
ax_3d.view_init(elev=20, azim=45)  # Adjust elevation and azimuth to improve visibility

ax_3d.set_title(f'3D Phase Space')
ax_3d.legend()


plt.tight_layout()
plt.title("Simulating the NF-kB network")
# why does the amplitude og NFKB keep changing?
def multi_amplitude_plot(A_min, A_max, stepsize):
    
    Amp_arr = np.linspace(A_min, A_max, stepsize)

    all_sims = []

    for i in range(stepsize):
        print("Simulation nr.: ", i)
        TNF_simi, ome, T_TNFi = TNF_sin_osc(no_osc, T_intt, Amp_arr[i], omega, *p0)
        ti, N_ni, I_mi, Ii, IKK_ai, IKK_ii, TNFi = np.array(TNF_simi["t"]), np.array(TNF_simi["N_n"]), np.array(TNF_simi["I_m"]), np.array(TNF_simi["I"]), np.array(TNF_simi["IKK_a"]), np.array(TNF_simi["IKK_i"]), np.array(TNF_simi["TNF"])
        all_sims.append([ti, N_ni, TNFi, I_mi, Ii, IKK_ai, IKK_ii])
    
    return all_sims
A_min = 0.21
A_max = 0.225
stepsize = 30

sims = multi_amplitude_plot(A_min, A_max, stepsize)
Amp_arr = np.linspace(A_min, A_max, stepsize)
all_t = [sim[0] for sim in sims]
all_N_ni = [sim[1] for sim in sims]
all_TNF = [sim[2] for sim in sims]
all_Im = [sim[3] for sim in sims]
all_I = [sim[4] for sim in sims]
all_IKKa = [sim[5] for sim in sims]
all_IKK = [sim[6] for sim in sims]
Amp_arr
figure, axis = plt.subplots(stepsize, figsize=[10,stepsize*5])

for i in tqdm(range(stepsize)):
        axis[i].plot(all_t[i], x*all_N_ni[i], color="#FF0000", alpha=0.8, lw=0.5, label = f'{x}x  NF$\kappa$B')
        axis[i].plot(all_t[i], all_TNF[i], alpha=1, color="#03FF00", lw=0.5, label = f'TNF with T = {T_TNF2:.4}, A = {Amp_arr[i]}')
        axis[i].set_title(rf'(Sim. Nr.: {i}) Sinusoidal oscillation: $\Omega$ = {omega}/1, A={Amp_arr[i]}')
        axis[i].legend();  
for ax in axis.flat:
    ax.set(xlabel='time [min.]', ylabel='concentration')
for ax in axis.flat:
    ax.label_outer()

    plt.tight_layout()
plt.figure(figsize=(12,6))
plt.plot(all_t[4][:100000], x*all_N_ni[4][:100000], color="#FF0000", alpha=1, lw=0.5, label = f'{x}x  NF$\kappa$B')
plt.plot(all_t[4][:100000], all_TNF[4][:100000], alpha=1, color="#03FF00", lw=0.5, label = f'TNF with T = {T_TNF2:.4}, A = {Amp_arr[i]}')
plt.title(rf'Sinusoidal oscillation: $\Omega$ = {omega}/1, A={Amp_arr[i]}')
plt.legend();  
plt.figure(figsize=(12,6))
plt.plot(all_t[4], x*all_N_ni[4], color="#FF0000", alpha=0.8, lw=0.5, label = f'{x}x  NF$\kappa$B')
plt.plot(all_t[4], all_TNF[4], alpha=1, color="#03FF00", lw=0.5, label = f'TNF with T = {T_TNF2:.4}, A = {Amp_arr[i]}')
plt.title(rf'Sinusoidal oscillation: $\Omega$ = {omega}/1, A={Amp_arr[i]}')
plt.legend();  
fig = plt.figure(figsize=(6, 6), dpi=200)
ax_3d = fig.add_subplot(111, projection='3d')  # Correct index for 3D plot
ax_3d.plot(all_Im[-1][1500:], all_I[-1][1500:], all_N_ni[-1][1500:] * 5, color="#FF0000", alpha=0.8, lw=0.5, label=f'System Trajectory with Omega = {omega}/1, A = {Amp_arr[-1]}')
ax_3d.set_xlabel(r'IkB_m', labelpad=10)
ax_3d.set_ylabel('IkB', labelpad=10)
# Adjust the z-label position
ax_3d.set_zlabel('5x NFkB', labelpad=10)  # Adjust labelpad as needed

# Adjusting the view angle
ax_3d.view_init(elev=20, azim=60)  # Adjust elevation and azimuth to improve visibility

ax_3d.set_title(f'3D Phase Space')
ax_3d.legend()


plt.tight_layout()
plt.title("Simulating the NF-kB network")
for i in range(stepsize):
    fig = plt.figure(figsize=(6, 6), dpi=200)
    ax_3d = fig.add_subplot(111, projection='3d')  # Correct index for 3D plot
    ax_3d.plot(all_Im[i][1500:], all_I[i][1500:], all_N_ni[i][1500:] * 5, color="#FF0000", alpha=0.8, lw=0.5, label=f'System Trajectory with Omega = {omega}/1, A = {Amp_arr[i]}')
    ax_3d.set_xlabel(r'IkB_m', labelpad=10)
    ax_3d.set_ylabel('IkB', labelpad=10)
    # Adjust the z-label position
    ax_3d.set_zlabel('5x NFkB', labelpad=10)  # Adjust labelpad as needed

    # Adjusting the view angle
    ax_3d.view_init(elev=20, azim=75)  # Adjust elevation and azimuth to improve visibility

    ax_3d.set_title(f'3D Phase Space')
    ax_3d.legend()


    plt.tight_layout()
    plt.title("Simulating the NF-kB network")
